## Population Model

In [ ]:
# Adjustables
is_sim = True
noise_mult = 1e-2
drug = 'small_noise'+str(noise_mult)+'_many_timepoints'
num_PK_comp = 2
observation_name = 'Platelets '

In [ ]:
# Set folders to save to
image_file = "../Images/parameter_inference/"
if is_sim:
    PK_data_file = "../Data_and_parameters/PK_sim/"
    PD_data_file = "../Data_and_parameters/PD_sim/"
else:
    PK_data_file = "../Data_and_parameters/PK_real/"
    PD_data_file = "../Data_and_parameters/PD_real/"

### PK Analysis

In [ ]:
# import pandas
# import numpy as np

# # Load in the data simulated from the Naive model
# df = pandas.read_csv("../Data_and_parameters/PK_sim/sythesised_data_real_timepoints.csv")
# df = df.sort_values(['ID', 'TIME'], ascending=True, ignore_index=True)
# dose_unit = "mg"

# # Reformat this into Something Chi understands
# df = df.rename(columns={'TIME':'Time', 'OBS': 'Value', 'DOSE':'Dose group'})
# df['Observable'] = ['central.drug_concentration']*len(df)
# dosing_df = df.groupby('ID',as_index=False)['Dose group'].mean()
# dosing_df.columns = ['ID', 'Dose']
# dosing_df['Time'] = [0.0]*len(dosing_df)
# dosing_df['Duration'] = [0.001]*len(dosing_df)

# df = pandas.concat([df, dosing_df], join='outer', ignore_index=True)
# df

In [ ]:
# Define the Data-generating parameters
import pandas
import numpy as np
from plotly import figure_factory as ff

PK_actual_params = np.load(PK_data_file+"actual_params.npy")

PK_param_names = PK_actual_params[0, :]
PK_actual_params = PK_actual_params[1, :].astype('float64')

# Define population parameters
PK_actual_pop_params = np.load(PK_data_file+"actual_pop_params.npy")
PK_actual_pop_params[1, -1] = noise_mult*float(PK_actual_pop_params[1, -1])
table_df = pandas.DataFrame(PK_actual_pop_params.transpose(), columns=['Parameter name', 'Data generating value'])
table_df = table_df.astype({'Data generating value':float})
table_df = table_df.round({'Data generating value':6})
table_df = table_df.set_index('Parameter name').transpose()
pop_param_names = PK_actual_pop_params[0, :]
PK_actual_pop_params = PK_actual_pop_params[1, :].astype('float64')

fig =  ff.create_table(table_df)
fig.update_layout(
    width=500,
    height=45,
)
fig.write_image(image_file + "PK_"+drug+"_data_table.svg")
fig.show()

In [ ]:
from Code.PK_model import ChiPKLin
import chi
import logging

# Remove Annoying logging
logger = logging.getLogger()
logger.handlers = []

# Set up the model
PK_model = ChiPKLin(num_comp=num_PK_comp)
PK_model.set_administration(
    compartment='central', amount_var='drug_amount')
noise_model = chi.MultiplicativeGaussianErrorModel()
population_model = chi.ComposedPopulationModel([
    chi.PooledModel(n_dim=1, dim_names=[PK_param_names[0]]), # Clearance - Pooled model
    chi.LogNormalModel(n_dim=1, dim_names=[PK_param_names[1]]), # Central volume - Mixed Effects model
    chi.PooledModel(n_dim=2*(num_PK_comp-1)+1, dim_names=list(PK_param_names[2:])) # Periferal compartment and noise parameters - Pooled model
])

# Set up the inference problem
problem = chi.ProblemModellingController(PK_model, noise_model)
problem.set_population_model(population_model)

# Set the bounds on the parameters and start point
# lower_bound = [0.01, 0.1*V_c_approx, 0.01, 0.01*V_c_approx, 0.0001]
# upper_bound = [100, 10*V_c_approx, 100, 100*V_c_approx, 1]

# np.save("../Data_and_parameters/PK_sim/bounds", np.asarray([lower_bound, upper_bound]))
PK_bounds = np.load("../Data_and_parameters/PK_sim/bounds.npy")



In [ ]:
# Create simulated data
dose_amts = [1.0, 2.0, 3.0]
n_ids_data = 15
pop_predictive_model = problem.get_predictive_model()

times = np.arange(0.05, 5, 0.1)[:]
# data = pandas.DataFrame(columns=["ID", "Time", "Observable", "Value", "Dose group", "Duration", "Dose"])

# for i, dose in enumerate(dose_amts):
#     # Set administration and dosing regimen
#     pop_predictive_model.set_dosing_regimen(dose=dose, period=0)
#     patient_data = pop_predictive_model.sample(
#         PK_actual_pop_params, times, include_regimen=True, n_samples=n_ids_data)
#     patient_data["ID"] = patient_data["ID"] + n_ids_data*i
#     patient_data["Dose group"] = dose
#     data = pandas.concat([data, patient_data])

# data.to_csv(PK_data_file + drug + "_data.csv", index=False)
df = pandas.read_csv(PK_data_file + drug + "_data.csv")
dose_unit = "mg"
n_ids_data = int(len(df["ID"].unique())/len(df["Dose group"].unique()))

problem.set_data(df)
df

In [ ]:
# First estimate the parameter V_c. We can do this by drawing a line through the first 2 points and seeing where it crosses the y-axis
df_obs = df.dropna(subset=['Observable'], inplace=False)
df_dose = np.asarray(df.groupby('ID')['Dose'].first())
df_obs['rank'] = df_obs.sort_values('Time').groupby('ID').cumcount()+1
first_points = df_obs[df_obs['rank'] == 1].sort_values('ID', ignore_index=True)
second_points = df_obs[df_obs['rank'] == 2].sort_values('ID', ignore_index=True)

y_0 = first_points['Value'] - first_points['Time'] * (
    (first_points['Value'] - second_points['Value'])
    / (first_points['Time'] - second_points['Time'])
)
V_c_approx = (df_dose/y_0).mean()
print("Aproximate "+PK_param_names[1]+": "+str(V_c_approx))

In [ ]:
import pints

# Set Log prior
log_prior = pints.ComposedLogPrior(
    pints.UniformLogPrior(PK_bounds[0,0], PK_bounds[1,0]),
    pints.UniformLogPrior(np.log(PK_bounds[0,1]), np.log(PK_bounds[1,1])),      # Log mean of central volume
    pints.LogNormalLogPrior(-1, 0.4),   # Log std. of central volume
    pints.UniformLogPrior(PK_bounds[0,2:-1], PK_bounds[1,2:-1]),
    pints.LogNormalLogPrior(-1, 0.4)
)
problem.set_log_prior(log_prior)

# Define hierarchical log-posterior
log_posterior = problem.get_log_posterior()

# Transform Parameter Space
transformation = pints.ComposedTransformation(
    pints.LogTransformation(len(df['ID'].unique())),
    pints.RectangularBoundariesTransformation(PK_bounds[0,0], PK_bounds[1,0]),
    pints.RectangularBoundariesTransformation(np.log(PK_bounds[0,1]), np.log(PK_bounds[1,1])),
    pints.IdentityTransformation(1),
    pints.RectangularBoundariesTransformation(PK_bounds[0,2:-1], PK_bounds[1,2:-1]),
    pints.IdentityTransformation(1),
)


<!-- import numpy as np
import pints
import chi
from Code.PK_model import ChiPKLin
import logging

# Remove Annoying logging
logger = logging.getLogger()
logger.handlers = []

# Define parameters
PK_params = np.load("../Data_and_parameters/PK_sim/actual_params.npy")
PK_params = PK_params[:, [1,0,3,2, -1]]
PK_param_names = PK_params[0, :]
PK_params = PK_params[1, :].astype('float64')

# Define population parameters
population_parameters = np.concatenate(([PK_params[0], np.log(PK_params[1]), 0.3], PK_params[2:]))

# Define pharmacokinetic model
mechanistic_model = ChiPKLin(num_comp=2)
mechanistic_model.set_administration(
    compartment='central', amount_var='drug_amount')
error_model = chi.MultiplicativeGaussianErrorModel()

# Define population model

# pop_theta_names = PK_param_names[1].split(",")
# pop_typ_name = pop_theta_names[0]+"_typ,"+pop_theta_names[1]
# pop_omega_name = "omega_"+pop_theta_names[0]

population_model = chi.ComposedPopulationModel([
    chi.PooledModel(n_dim=1, dim_names=[PK_param_names[0]]),
    chi.LogNormalModel(
        n_dim=1, dim_names=[PK_param_names[1]]
    ),
    chi.PooledModel(n_dim=3, dim_names=list(PK_param_names[2:]))])
pop_predictive_model = chi.PopulationPredictiveModel(
    chi.PredictiveModel(mechanistic_model, error_model), population_model)

# Get individual parameters
dose_amts = [1, 2, 4]
n_ids = 12 * len(dose_amts)
individual_parameters = population_model.sample(
    parameters=population_parameters,
    n_samples=n_ids,
    seed=1
)

times = np.arange(0.1, 5.1, 0.5)
data = pandas.DataFrame(columns=["ID", "Time", "Observable", "Value", "Duration", "Dose"])

for i, dose in enumerate(dose_amts):
    # Set administration and dosing regimen
    pop_predictive_model.set_dosing_regimen(dose=dose, period=0)
    patient_data = pop_predictive_model.sample(
        population_parameters, times, include_regimen=True, n_samples=12)
    patient_data["ID"] = patient_data["ID"] + 12*i
    data = pandas.concat([data, patient_data])

problem = chi.ProblemModellingController(mechanistic_model, error_model)
problem.set_population_model(population_model)
problem.set_data(data)
data -->

#### Visualisation

In [ ]:
# Adjust the DataFrame for use in figures
dose_amts = df.Dose.unique()
dose_amts = dose_amts[~np.isnan(dose_amts)]
dose_groups = df["Dose group"].unique()
x_label = "Time, hours"
y_label = "Drug concentration, mg/L"
col_label = 'Dose, '+dose_unit
df_graph = df.rename(columns={
    'Dose group':col_label, 
    'Time':x_label, 
    'Value':y_label
}, errors="raise")
df_graph = df_graph.astype({col_label: 'category', 'ID': 'category'}, errors="raise")

In [ ]:
# Visualise population model (measurement space)
n_ids = 1000
n_times = 1000

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express.colors as pxclrs

fig = go.Figure()

ind_colour_selection = np.asarray([
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0, high=0.23),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0.35, high=0.65),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0.7, high=1.0)
])

individual_parameters = population_model.sample(
    parameters=PK_actual_pop_params,
    n_samples=n_ids,
    seed=1
)
more_times = np.linspace(start=0, stop=5, num=n_times)

for i, group in enumerate(dose_groups):
    # Simulate population distribution of measurements
    measurements = df.loc[(df["Dose group"]==group)]
    dose = measurements["Dose"].dropna().mean()

    pop_measurements = np.empty(shape=(n_ids, n_times))
    PK_model.set_dosing_regimen(dose=dose, period=0)
    
    for j, patient in enumerate(df_graph.loc[df_graph[col_label]==group]['ID'].unique()):
        fig.add_trace(go.Scatter(
            x=df_graph.loc[df_graph['ID']==patient][x_label],
            y=df_graph.loc[df_graph['ID']==patient][y_label],
            mode='markers',
            marker_color=ind_colour_selection[i, j],
            name='Simulated data',
            legendgroup = dose,
            showlegend = j==int(n_ids_data/2),
            legendgrouptitle = {'text': 'Dose '+str(dose)+' '+dose_unit}
        ))

    # Plot 5th to 95th percentile of population distribution
    for idd, patient_parameters in enumerate(individual_parameters):
        result = PK_model.simulate(patient_parameters[:-1], more_times)[0]
        pop_measurements[idd] = noise_model.sample(
            patient_parameters[-1:], result)[:, 0]

    fifth = np.percentile(pop_measurements, q=5, axis=0)
    ninety_fifth = np.percentile(pop_measurements, q=95, axis=0)
    fig.add_trace(go.Scatter(
        x=np.hstack([more_times, more_times[::-1]]),
        y=np.hstack([fifth, ninety_fifth[::-1]]),
        line=dict(width=1, color=ind_colour_selection[i, int(n_ids_data/2)]),
        fill='toself',
        name='Population model',
        text=r"90% bulk probability",
        hoverinfo='text',
        showlegend=True
    ))

fig.update_layout(
    xaxis_title=x_label,
    yaxis_title=y_label,
    template='plotly_white',
    width=750,
    height=500,
)
fig.update_yaxes(type="log", minor=dict(dtick='D1', showgrid=True), dtick = 1)
fig.write_image(image_file + "PK_"+drug+"_population_data.svg")
fig.show()

#### Maximum Log likelihood method

In [ ]:
from plotly import figure_factory as ff
from Code.Inference import OptimisationController

# Set the start point for optimisation
start_point = np.exp((np.log(PK_bounds[0])+np.log(PK_bounds[1]))*0.5)
start_point[1] = np.log(V_c_approx) # Log of V_c mean
start_point = np.insert(start_point, 2, 0.15) # V_c std.

# Optimise the model with respect to the data
optimisation = OptimisationController(
    log_posterior,
    # start_point,
    # method=pints.CMAES,
    # boundaries=pints.RectangularBoundaries(PK_bounds[0], PK_bounds[1])
)
optimisation.set_n_runs(1)
optimisation.set_initial_point([1], [start_point])
optimisation.set_optimiser(pints.CMAES)
optimisation.set_transform(transformation)
result = optimisation.run(show_run_progress_bar=False, log_to_screen=False)

# Show summary of optimisation
print('Log-Likelihood Value: \t'+str(result['Score'].iloc[0]))
time = result['Time'].iloc[0]
print('Time Taken: \t'+str(int(time/60))+" minutes, "+str(int(time%60))+" seconds, ")

print('Result:')
parameters = (result.loc[result['ID'].isnull()])['Estimate'].values
summary_data = np.transpose(np.array([pop_param_names, parameters, PK_actual_pop_params]))
summary_df = pandas.DataFrame(summary_data, columns = ['Parameter', 'Optimised', 'True'])

summary_df.to_csv(PK_data_file+drug+"_opt_pop_results.csv", index=False)

opt_ind_params = np.asarray(result.dropna(subset=['ID'])['Estimate'])
np.save(PK_data_file+drug+"_opt_ind_results.npy", opt_ind_params)

summary_df = pandas.read_csv(PK_data_file+drug+"_opt_pop_results.csv")
table_df = summary_df.round({'Optimised':4, 'True':4})
table_df = table_df.set_index('Parameter').transpose()
fig =  ff.create_table(table_df, index=True)
fig.update_layout(
    width=525,
    height=65,
)
fig.write_image(image_file + "PK_"+drug+"_opt_table.svg")
fig.show()

In [ ]:
summary_df = pandas.read_csv(PK_data_file+drug+"_opt_pop_results.csv")
opt_ind_params = np.load(PK_data_file+drug+"_opt_ind_results.npy")

In [ ]:
# Visualise the results
time_span = df["Time"].max()
n_times = 1000
more_times = np.linspace(0, time_span, n_times)

inferred_pop_params = np.asarray(summary_df['Optimised'])
avg_inferred = np.delete(inferred_pop_params, 2)
avg_inferred[1] = np.exp(avg_inferred[1])


n_ids = 1000
sim_ind_params = population_model.sample(
    parameters=inferred_pop_params,
    n_samples=n_ids
)

# Plot the Observations
fig = go.Figure()

# Plot the population model for each dose
for i, group in enumerate(dose_groups):
    # Simulate and Plot from the population maximum likelihood estimation
    amt = dose_amts[i]
    PK_model.set_dosing_regimen(amt)
    more_values = PK_model.simulate(avg_inferred[:-1], more_times)
    fig.add_trace(
        go.Scatter(
            x=more_times,
            y=more_values[0],
            mode='lines',
            line=dict(color=ind_colour_selection[i, int(n_ids_data/2)], dash='dash'),
            name='Typical simulation',
            legendgroup= group,
            legendgrouptitle = {'text': 'Dose '+str(group)+' '+dose_unit},
            showlegend=True
        ), 
    )
    for j, patient in enumerate(df_graph.loc[df_graph[col_label]==group]['ID'].unique()):
        PK_model.set_dosing_regimen(df_graph.loc[(df_graph['ID']==patient) & df_graph['Observable'].isna()]['Dose'].iat[0])
        individual_parameters = avg_inferred
        individual_parameters[1] = opt_ind_params[i*n_ids_data + j]
        individual_values = PK_model.simulate(individual_parameters[:-1], more_times)
        fig.add_trace(
            go.Scatter(
                x=more_times,
                y=individual_values[0],
                name='Individual simulation',
                legendgroup = group,
                showlegend= j==int(n_ids_data/2),
                mode="lines",
                line=go.scatter.Line(color=ind_colour_selection[i, j], width=0.5),
                opacity=0.5,
            ),
        )
        fig.add_trace(go.Scatter(
            x=df_graph.loc[df_graph['ID']==patient][x_label],
            y=df_graph.loc[df_graph['ID']==patient][y_label],
            mode='markers',
            marker_color=ind_colour_selection[i, j],
            name='Data',
            legendgroup = group,
            showlegend = j==int(n_ids_data/2),
        ))
    
    PK_model.set_dosing_regimen(amt)
    # Find 5th to 95th percentile of population distribution
    simulations = np.empty(shape=(n_ids, n_times))
    for idd, patient_parameters in enumerate(sim_ind_params):
        ind_sim = PK_model.simulate(patient_parameters[:-1], more_times)[0]
        simulations[idd] = noise_model.sample(
            patient_parameters[-1:], ind_sim)[:, 0]
    fifth = np.percentile(simulations, q=5, axis=0)
    ninety_fifth = np.percentile(simulations, q=95, axis=0)

    # Plot the variability
    fig.add_trace(go.Scatter(
        x=np.hstack([more_times, more_times[::-1]]),
        y=np.hstack([fifth, ninety_fifth[::-1]]),
        line=dict(width=0, color=ind_colour_selection[i, int(n_ids_data/2)]),
        fill='toself',
        name='Population model',
        text=r"90% bulk probability",
        hoverinfo='text',
        legendgroup= group,
        showlegend=True
    ))

fig.update_layout(
    xaxis_title=x_label,
    yaxis_title=y_label,
    template='plotly_white',
    width=750,
    height=500,
)
y_range = [np.log10(np.min(df_graph[y_label]))-0.05, np.log10(np.max(more_values))+0.2]
x_range = [0, 5]
fig['layout']['yaxis'].update(type="log", minor=dict(dtick='D1', showgrid=True), dtick = 1, range=y_range)
fig['layout']['xaxis'].update(range=x_range)
fig.write_image(image_file + "PK_"+drug+"_opt_graph.svg")
fig.show()

#### Bayesian Inference

In [ ]:
# Further adjustables
num_iterations = 6000  # max iterations per parameter
num_samples = 200  # number of wanted final samples per run
n_runs = 5

In [ ]:
inferred_pop_params = pandas.read_csv(PK_data_file+drug+"_opt_pop_results.csv")
opt_params = inferred_pop_params['Optimised']
opt_ind_params = np.load(PK_data_file+drug+"_opt_ind_results.npy")

pop_ranges = np.asarray([
    np.concatenate((PK_bounds[0,0:1], [np.log(PK_bounds[0,1])], [np.exp(-1-2*0.4)], PK_bounds[0,2:])),
    np.concatenate((PK_bounds[1,0:1], [np.log(PK_bounds[1,1])], [np.exp(-1+2*0.4)], PK_bounds[1,2:]))
])
start_pop_points = np.concatenate((
    np.linspace(start=opt_params, stop=pop_ranges[0], num=n_runs-1, endpoint=False)[:int(0.5*(n_runs-1))], 
    np.linspace(start=opt_params, stop=pop_ranges[1], num=n_runs-1, endpoint=False)[:int(0.5*(n_runs-1))], 
))
# print(sampler._initial_params)
# start_points = optimisation.make_initial_point(3, start_pop_points)
# start_points = np.concatenate((
#     start_points, optimisation.make_initial_point(1, opt_params, opt_ind_params)
# ))

In [ ]:
# from Code.Inference import SamplingController
from chi._inference import SamplingController

sampler = SamplingController(log_posterior)
sampler.set_n_runs(n_runs)
# if n_runs%2==0:
#     sampler.set_initial_point(range(1, n_runs), start_pop_points[1:, :])
# else:
#     sampler.set_initial_point(range(1, n_runs), start_pop_points)
# sampler.set_initial_point([n_runs], opt_params, opt_ind_params)

cov_0 = 0.01 * np.abs(np.concatenate((opt_ind_params, opt_params)))
# cov_0 = None
sampler.set_sampler(pints.NoUTurnMCMC)
# sampler.set_stop_criterion(max_iterations= num_iterations*len(opt_params), r_hat=1.01) # num_iterations*len(opt_params)
sampler.set_transform(transformation)
posterior_samples = sampler.run(n_iterations=num_samples+70, log_to_screen=True) #, sigma0=cov_0)

In [ ]:
import xarray as xr
# posterior_samples.to_netcdf(PK_data_file+drug+"_MCMC_samples.nc")
posterior_samples = xr.open_dataset(PK_data_file+drug+"_MCMC_samples.nc").load()

In [ ]:
posterior_samples

In [ ]:
import arviz as az
import matplotlib.pyplot as plt

az.style.use(["arviz-white",  "arviz-viridish"])

lines = list(zip(*(population_model.get_parameter_names(),[{}]*len(PK_actual_pop_params), PK_actual_pop_params)))
az.plot_trace(posterior_samples.drop_isel(draw=range(0, 100)), var_names=('^(?!.*?central.V_c).*'), filter_vars="regex", lines=lines, divergences='bottom')
plt.savefig(image_file +"PK_"+drug+"MCMC_Pop_trace_plot.svg")
plt.show()

In [ ]:
def V_c_transformation(V_c_params):
    trans_V_c = V_c_params
    trans_V_c['Log mean central.V_c'] = np.exp(trans_V_c['Log mean central.V_c'])
    return trans_V_c
lines[1] = (population_model.get_parameter_names()[1], {}, np.exp(PK_actual_pop_params[1]))
lines.append(("central.V_c", {}, np.exp(PK_actual_pop_params[1])))
az.plot_trace(posterior_samples.drop_isel(draw=range(0, 100)), var_names=('.*central.V_c'), filter_vars="regex", lines=lines, transform=V_c_transformation)
plt.savefig(image_file +"PK_"+drug+"MCMC_ind_trace_plot.svg")
plt.show()

In [ ]:
import plotly.express.colors as pxclrs
from plotly.subplots import make_subplots
import plotly.graph_objects as go

colour_selection = np.asarray([
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0, high=0.2),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0.8, high=1.0),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0.4, high=0.6),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("turbo"), n_ids_data, low=0.8, high=1.0)
])
dose_group_colours = dict(zip(df["Dose group"].unique(), colour_selection[:len(df["Dose group"].unique())]))

# Discard warmup iterations
posterior_samples = posterior_samples.sel(draw=slice(-num_samples, None))

# Visualise posteriors
fig = make_subplots(rows=2, cols=2, shared_yaxes=True)
fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Pooled central.K_cl'].values.flatten(),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=1,
    col=1
)
# fig.add_trace(
#     go.Scatter(
#         name='Data-generating parameters',
#         x=[np.exp(population_parameters[0])]*2,
#         y=[0, 0.08],
#         mode='lines',
#         line_color='black',
#         showlegend=False
#     ),
#     row=1,
#     col=1
# )

fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Pooled periferal_1.K_p'].values.flatten(),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=1,
    col=2
)
# fig.add_trace(
#     go.Scatter(
#         name='Data-generating parameters',
#         x=[np.exp(population_parameters[3])]*2,
#         y=[0, 0.08],
#         mode='lines',
#         line_color='black',
#         showlegend=False
#     ),
#     row=2,
#     col=1
# )

fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Pooled periferal_1.V_p'].values.flatten(),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=2,
    col=1
)
# fig.add_trace(
#     go.Scatter(
#         name='Data-generating parameters',
#         x=[np.exp(population_parameters[4])]*2,
#         y=[0, 0.06],
#         mode='lines',
#         line_color='black',
#         showlegend=False
#     ),
#     row=2,
#     col=2
# )


fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Pooled Sigma rel.'].values.flatten(),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=2,
    col=2
)
# fig.add_trace(
#     go.Scatter(
#         name='Data-generating parameters',
#         x=[np.exp(population_parameters[5])]*2,
#         y=[0, 0.06],
#         mode='lines',
#         line_color='black',
#         showlegend=False
#     ),
#     row=3,
#     col=1
# )

fig.update_layout(
    xaxis_title='Clearance rate',
    # xaxis2_title='Central compartment volume',
    xaxis2_title='Compartment transfer rate',
    xaxis3_title='Periferal compartment volume',
    xaxis4_title='Sigma_m',
    yaxis_title='Probability',
    yaxis3_title='Probability',
    # yaxis5_title='Probability',
    template='plotly_white',
    bargap=0,
    bargroupgap=0,
    width=900, 
    height=540,
    barmode='overlay'
)
fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=2, shared_yaxes=True, shared_xaxes=True)

for i, patient in enumerate(df["ID"].unique()):
    group = df.loc[df["ID"]==patient, "Dose group"].iloc[0]
    colour = dose_group_colours[group][i%n_ids_data]
    fig.add_trace(
        go.Histogram(
            name='Patient '+ str(patient)+' Post. samples',
            x=posterior_samples['central.V_c'].values[
                0, :, i].flatten(),
            histnorm='probability',
            showlegend=False,
            marker=dict(color=colour, opacity=0.7)
        ),
        row=1,
        col=1
    )

# fig.add_trace(
#     go.Scatter(
#         name='Data-gen. Mean parameter',
#         x=[np.exp(PK_actual_pop_params[1])]*2,
#         y=[0, 0.1],
#         mode='lines',
#         line_color='black',
#         showlegend=False
#     ),
#     row=1,
#     col=2
# )

fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=np.exp(posterior_samples['Log mean central.V_c'].values.flatten()),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=2,
    col=1
)
# fig.add_trace(
#     go.Scatter(
#         name='Data-generating parameters',
#         x=[PK_actual_pop_params[1]]*2,
#         y=[0, 0.08],
#         mode='lines',
#         line_color='black',
#         showlegend=False
#     ),
#     row=1,
#     col=2
# )

fig.add_trace(
    go.Histogram(
        name='Posterior samples',
        x=posterior_samples['Log std. central.V_c'].values.flatten(),
        histnorm='probability',
        marker_color='lightgrey',
        showlegend=False
    ),
    row=2,
    col=2
)
# fig.add_trace(
#     go.Scatter(
#         name='Data-generating parameters',
#         x=[PK_actual_pop_params[2]]*2,
#         y=[0, 0.08],
#         mode='lines',
#         line_color='black',
#         showlegend=False
#     ),
#     row=2,
#     col=2
# )

fig.update_layout(
    xaxis3_title='V_c',
    xaxis4_title='Log std. V_c',
    yaxis_title='Individual parameter probabilities',
    yaxis3_title='Population parameters probability',
    bargap=0,
    bargroupgap=0,
    width=900, 
    height=540,
    template='plotly_white'
)
fig.show()

In [ ]:
posterior_samples.close()

### PD Analysis

In [ ]:
# import pandas
# import numpy as np

# # Load in the data simulated from the Naive model
# df = pandas.read_csv("../Data_and_parameters/PK_sim/sythesised_data_real_timepoints.csv")
# df = df.sort_values(['ID', 'TIME'], ascending=True, ignore_index=True)
# dose_unit = "mg"

# # Reformat this into Something Chi understands
# df = df.rename(columns={'TIME':'Time', 'OBS': 'Value', 'DOSE':'Dose group'})
# df['Observable'] = ['central.drug_concentration']*len(df)
# dosing_df = df.groupby('ID',as_index=False)['Dose group'].mean()
# dosing_df.columns = ['ID', 'Dose']
# dosing_df['Time'] = [0.0]*len(dosing_df)
# dosing_df['Duration'] = [0.001]*len(dosing_df)

# df = pandas.concat([df, dosing_df], join='outer', ignore_index=True)
# df

In [ ]:
# Define the Data-generating parameters
import pandas
import numpy as np
import plotly.figure_factory as ff

PK_actual_params = np.load(PK_data_file+"actual_params.npy")
PD_actual_params = np.load(PD_data_file+"actual_params.npy")

# Define population parameters
PK_actual_pop_params = np.load(PK_data_file+"actual_pop_params.npy")
order = [6, 0, 1, 2, 3, 4, 7, 8, 9,  5, 10]
PKPD_actual_pop_params = np.concatenate((PK_actual_pop_params, PD_actual_params), axis=1)[:, order]

table_df = pandas.DataFrame(PKPD_actual_pop_params.transpose(), columns=['Parameter name', 'Data generating value'])
table_df = table_df.astype({'Data generating value':float})
table_df = table_df.round({'Data generating value':4})
table_df = table_df.set_index('Parameter name').transpose()

fig =  ff.create_table(table_df)
fig.update_layout(
    width=700,
    height=45,
)
fig.write_image(image_file + "PKPD_"+drug+"_data_table.svg")

pop_param_names = PKPD_actual_pop_params[0, :]
PKPD_actual_pop_params = PKPD_actual_pop_params[1, :].astype('float64')

PK_param_names = PK_actual_params[0, :]
PD_param_names = PD_actual_params[0, :]
PK_actual_params = PK_actual_params[1, :].astype('float64')
PD_actual_params = PD_actual_params[1, :].astype('float64')
fig.show()


In [ ]:
from Code.PD_model import ChiMyelotoxicityPKPD
from Code.PK_model import ChiPKLin
import chi
import logging

# Remove Annoying logging
logger = logging.getLogger()
logger.handlers = []

# Set up the model
PK_model = ChiPKLin(num_comp=num_PK_comp)
PKPD_model = ChiMyelotoxicityPKPD(PK_model, 'central.drug_concentration')

PKPD_model.set_administration(
    compartment='PK_central', amount_var='drug_amount')

# param_names = np.delete(pop_param_names[:-2], 3)
# param_names[2] = 'V_c, L'
# names = dict(zip(
#     np.concatenate((PKPD_model.parameters()[:6], PKPD_model.parameters()[7:])),
#     param_names
# ))
# PKPD_model.set_parameter_names(names)
PKPD_model.set_outputs(['PK_central.drug_concentration', 'circulating.R'])

PK_noise_model = chi.MultiplicativeGaussianErrorModel()
PD_noise_model = chi.GaussianErrorModel()

fixed = {"drug.drug_concentration": 0}
population_model = chi.ComposedPopulationModel([
    chi.PooledModel(n_dim=1), # E_drug - Pooled model
    chi.PooledModel(n_dim=1), # Clearance - Pooled model
    chi.LogNormalModel(n_dim=1), # Central volume - Mixed Effects model
    chi.PooledModel(n_dim=1), # R_0 - Pooled model
    # chi.PooledModel(n_dim=1), # Nonsense Parameter
    chi.PooledModel(n_dim=2*(num_PK_comp-1)), # Periferal compartment - Pooled model
    chi.PooledModel(n_dim=1), # gamma
    chi.PooledModel(n_dim=1), # MTT
    chi.PooledModel(n_dim=1, dim_names=[PK_param_names[-1]]), # PK Noise parameter
    chi.PooledModel(n_dim=1, dim_names=[PD_param_names[-3]]) # PD Noise parameter
])

# Set up the inference problem
problem = chi.ProblemModellingController(PKPD_model, [PK_noise_model, PD_noise_model])
problem.fix_parameters(fixed)
problem.set_population_model(population_model)

# fixed_population_model = chi.ReducedPopulationModel(population_model)
# fixed_population_model.fix_parameters(fixed)

# Set the bounds on the parameters and start point
# lower_bound = [0.01, 0.1*V_c_approx, 0.01, 0.01*V_c_approx, 0.0001]
# upper_bound = [100, 10*V_c_approx, 100, 100*V_c_approx, 1]

# np.save("../Data_and_parameters/PK_sim/bounds", np.asarray([lower_bound, upper_bound]))
PK_bounds = np.load("../Data_and_parameters/PK_sim/bounds.npy")
PD_bounds = np.load("../Data_and_parameters/PD_sim/bounds.npy")

PD_bounds[0] = 1.2*PD_bounds[0]
PD_bounds[1] = 0.6*PD_bounds[1]


In [ ]:
# Create simulated data
dose_amts = [1.0, 2.0, 3.0]
n_ids_data = 15
dose_time = 48
n_times_data = 50

PK_times = np.linspace(0.05, 5, n_times_data)[:] + dose_time
PD_times = np.linspace(-dose_time, 500, n_times_data)[:] + dose_time
# data = pandas.DataFrame(columns=["ID", "Time", "Observable", "Value", "Dose group", "Duration", "Dose"])

# # Acquire patient parameters
# individual_parameters = population_model.sample(
#     parameters=PKPD_actual_pop_params,
#     n_samples=n_ids_data*len(dose_amts)
# )
# # for i, var in enumerate(PKPD_model._const_names):
# #     print(var, individual_parameters[0, i])

# for i, dose in enumerate(dose_amts):
#     # Set administration and dosing regimen
#     PKPD_model.set_dosing_regimen(dose=dose, start=dose_time, period=0)
#     for patient in range(0, n_ids_data):
#         # Simulate model
#         param = np.insert(individual_parameters[n_ids_data*i + patient, :], 6, 0)
#         patient_result = PKPD_model.simulate(param[:-2], np.concatenate((PK_times, PD_times[5:])))
        
#         # produce data
#         include_PK_data = dose!=0.0

#         patient_data = pandas.DataFrame()
#         patient_data_length = len(PK_times)*include_PK_data+len(PD_times)+1
#         patient_data["ID"] = [n_ids_data*i + patient+1]*patient_data_length
#         if include_PK_data:
#             patient_data["Time"] = np.concatenate((PK_times, PD_times, [dose_time]))
#             patient_data["Observable"] = (
#                 ['PK_central.drug_concentration']*len(PK_times)+['circulating.R']*len(PD_times)+[None]
#             )
#             PK_result = PK_noise_model.sample(param[-2:-1], patient_result[0, :len(PK_times)])[:, 0]
#         else:
#             patient_data["Time"] = np.concatenate((PD_times, [dose_time]))
#             patient_data["Observable"] = (
#                 ['circulating.R']*len(PD_times)+[None]
#             )
#             PK_result = []

#         PD_result = PD_noise_model.sample(
#             param[-1:],
#             np.concatenate(([param[5]]*5, patient_result[1, len(PK_times):]))
#         )[:, 0]
#         patient_data["Value"] = np.concatenate((PK_result, PD_result, [None]))

#         patient_data["Dose group"] = [dose]*patient_data_length
#         patient_data["Duration"] = [None]*(patient_data_length-1)+[0.01]
#         patient_data["Dose"] = [None]*(patient_data_length-1)+[dose]
#         data = pandas.concat([data, patient_data])

# data.to_csv(PD_data_file + drug + "_data.csv", index=False)
# np.save(PD_data_file + drug +"_ind_Vc_param.npy", individual_parameters[:, 2])

df = pandas.read_csv(PD_data_file + drug + "_data.csv")
n_ids_data = int(len(df["ID"].unique())/len(df["Dose group"].unique()))

individual_parameters = np.asarray([PKPD_actual_pop_params]*(n_ids_data*len(dose_amts)))
individual_parameters[:, 2] = np.load(PD_data_file + drug +"_ind_Vc_param.npy")
dose_unit = "mg"

problem.set_data(df)
df

In [ ]:
from scipy.integrate import simpson
from scipy.stats import linregress

# First estimate the parameter V_c. We can do this by drawing a line through the first 2 points and seeing where it crosses the y-axis
df_obs = df.loc[df['Observable']=='PK_central.drug_concentration']
PK_data = df.loc[df['ID'].isin(df_obs['ID'].unique())]
df_dose = np.asarray(PK_data.groupby('ID')['Dose'].first())
df_obs['rank'] = df_obs.sort_values('Time').groupby('ID').cumcount()+1
first_points = df_obs[df_obs['rank'] == 1].sort_values('ID', ignore_index=True)
second_points = df_obs[df_obs['rank'] == 2].sort_values('ID', ignore_index=True)

y_0 = first_points['Value'] - (first_points['Time']-dose_time) * (
    (first_points['Value'] - second_points['Value'])
    / ((first_points['Time']-dose_time)- (second_points['Time']-dose_time))
)
V_c_approx = (df_dose/y_0).mean()

AUC_0_last = np.empty(len(PK_data["ID"].unique()))
C_last = np.empty(len(PK_data["ID"].unique()))
lambda_z = np.empty(len(PK_data["ID"].unique()))
for i, patient in enumerate(PK_data["ID"].unique()):
    y_ind = np.asarray(df_obs.loc[df_obs['ID']==patient]['Value'])
    x_ind = np.asarray(df_obs.loc[df_obs['ID']==patient]['Time'])-dose_time
    AUC_0_last[i] = simpson(y=y_ind, x=x_ind)
    C_last[i] = y_ind[-1]
    lambda_z[i] = linregress(x=x_ind[int(0.5*len(x_ind)):], y=x_ind[int(0.5*len(x_ind)):]).slope
AUC_inf = AUC_0_last+C_last/lambda_z
cl_approx = (df_dose/AUC_inf).mean()

df_R_before_dose = df[(df["Time"] < dose_time) & (df["Observable"]=='circulating.R')]
R_0_approx = np.mean(df_R_before_dose["Value"])

nadir = np.empty((2, len(PK_data["ID"].unique())))
for i, patient in enumerate(df["ID"].unique()):
    df_PD_ind = df[(df["ID"]==patient) & (df["Observable"]=='circulating.R')]
    df_nadir = df_PD_ind[df_PD_ind.Value == df_PD_ind.Value.min()]
    y_nadir = df_nadir.Value.iat[0]
    t_nadir = df_nadir.Time.iat[0]-dose_time
    nadir[:, i] = (t_nadir, y_nadir)

MTT_approx = nadir[0,:].mean()
table_df.reindex(["True", "Approximation"])
table_df.at["Approximate", 'R_0, 10^3/\\mu \\ textrm{L}'] = round(R_0_approx, 4)
table_df.at["Approximate", '\\log(V_{c, typ})'] = round(np.log(V_c_approx), 4)
table_df.at["Approximate", 'K_{cl}, L/hr'] = round(cl_approx, 4)
table_df.at["Approximate", 'MTT, \\ textrm{hr}'] = round(MTT_approx, 4)
fig =  ff.create_table(table_df, index=True)
fig.update_layout(
    width=1000,
    height=65
)
fig.write_image(image_file + "PKPD_"+drug+"_data_table.svg")
fig.show()

In [ ]:
pop_param_names

In [ ]:
import pints

log_prior = pints.ComposedLogPrior(
    pints.UniformLogPrior(PD_bounds[0,0], PD_bounds[1,0]),                  # S
    pints.LogNormalLogPrior(np.log(cl_approx), 0.75),                       # Clearance
    pints.GaussianLogPrior(np.log(V_c_approx), 0.4),                        # Log mean of central volume
    pints.LogNormalLogPrior(-1, 0.4),                                       # Log std. of central volume
    pints.LogNormalLogPrior(np.log(cl_approx), 3),                          # Periferal compartment transfer
    pints.LogNormalLogPrior(np.log(V_c_approx), 0.8),                       # Periferal compartment volume
    pints.LogNormalLogPrior(np.log(R_0_approx), 1),                         # R_0
    pints.LogNormalLogPrior(np.log(1), 0.1),                                # gamma
    pints.LogNormalLogPrior(np.log(MTT_approx), 0.8),                         # MTT
    pints.LogNormalLogPrior(-1, 0.4),                                       # PK Noise parameter
    pints.LogNormalLogPrior(np.log(0.1*R_0_approx), 0.4),                                       # PD Noise parameter
)
problem.set_log_prior(log_prior)

# Define hierarchical log-posterior
log_posterior = problem.get_log_posterior()


# Transform Parameter Space
# transformation = pints.ComposedTransformation(
#     pints.LogTransformation(len(df['ID'].unique())),                                            # Individual parameters
#     pints.RectangularBoundariesTransformation(PD_bounds[0,0], PD_bounds[1,0]),                  # S
#     pints.RectangularBoundariesTransformation(PK_bounds[0,0], PK_bounds[1,0]),                  # Clearance
#     pints.RectangularBoundariesTransformation(np.log(PK_bounds[0,1]), np.log(PK_bounds[1,1])),  # Log mean of central volume
#     pints.IdentityTransformation(1),                                                            # Log std. of central volume
#     pints.RectangularBoundariesTransformation(PK_bounds[0,2:-1], PK_bounds[1,2:-1]),            # Periferal compartment
#     pints.RectangularBoundariesTransformation(PD_bounds[0,1], PD_bounds[1,1]),                  # R_0
#     pints.RectangularBoundariesTransformation(PD_bounds[0,2], PD_bounds[1,2]),                  # gamma
#     pints.RectangularBoundariesTransformation(PD_bounds[0,3], PD_bounds[1,3]),                  # MTT
#     pints.RectangularBoundariesTransformation(PK_bounds[0,-1], PK_bounds[1,-1]),                # PK Noise parameter
#     pints.RectangularBoundariesTransformation(PD_bounds[0,-3], PD_bounds[1,-3])                 # PD Noise parameter
# )

<!-- import numpy as np
import pints
import chi
from Code.PK_model import ChiPKLin
import logging

# Remove Annoying logging
logger = logging.getLogger()
logger.handlers = []

# Define parameters
PK_params = np.load("../Data_and_parameters/PK_sim/actual_params.npy")
PK_params = PK_params[:, [1,0,3,2, -1]]
PK_param_names = PK_params[0, :]
PK_params = PK_params[1, :].astype('float64')

# Define population parameters
population_parameters = np.concatenate(([PK_params[0], np.log(PK_params[1]), 0.3], PK_params[2:]))

# Define pharmacokinetic model
mechanistic_model = ChiPKLin(num_comp=2)
mechanistic_model.set_administration(
    compartment='central', amount_var='drug_amount')
error_model = chi.MultiplicativeGaussianErrorModel()

# Define population model

# pop_theta_names = PK_param_names[1].split(",")
# pop_typ_name = pop_theta_names[0]+"_typ,"+pop_theta_names[1]
# pop_omega_name = "omega_"+pop_theta_names[0]

population_model = chi.ComposedPopulationModel([
    chi.PooledModel(n_dim=1, dim_names=[PK_param_names[0]]),
    chi.LogNormalModel(
        n_dim=1, dim_names=[PK_param_names[1]]
    ),
    chi.PooledModel(n_dim=3, dim_names=list(PK_param_names[2:]))])
pop_predictive_model = chi.PopulationPredictiveModel(
    chi.PredictiveModel(mechanistic_model, error_model), population_model)

# Get individual parameters
dose_amts = [1, 2, 4]
n_ids = 12 * len(dose_amts)
individual_parameters = population_model.sample(
    parameters=population_parameters,
    n_samples=n_ids,
    seed=1
)

times = np.arange(0.1, 5.1, 0.5)
data = pandas.DataFrame(columns=["ID", "Time", "Observable", "Value", "Duration", "Dose"])

for i, dose in enumerate(dose_amts):
    # Set administration and dosing regimen
    pop_predictive_model.set_dosing_regimen(dose=dose, period=0)
    patient_data = pop_predictive_model.sample(
        population_parameters, times, include_regimen=True, n_samples=12)
    patient_data["ID"] = patient_data["ID"] + 12*i
    data = pandas.concat([data, patient_data])

problem = chi.ProblemModellingController(mechanistic_model, error_model)
problem.set_population_model(population_model)
problem.set_data(data)
data -->

#### Visualisation

In [ ]:
# Adjust the DataFrame for use in figures
dose_amts = df.Dose.unique()
dose_amts = dose_amts[~np.isnan(dose_amts)]
dose_groups = df["Dose group"].unique()
x_label = "Time, hours"
col_label = 'Dose, '+dose_unit

PK_y_label = "Drug concentration, mg/L"
df_PK_graph = df.loc[df['Observable']=='PK_central.drug_concentration']
df_PK_graph = df_PK_graph.rename(columns={
    'Dose group':col_label, 
    'Time':x_label, 
    'Value':PK_y_label
}, errors="raise")
df_PK_graph = df_PK_graph.astype({col_label: 'category'}, errors="raise")


PD_y_label = "Blood cell concentration, 10^3/microL"
df_PD_graph = df.loc[df['Observable']=='circulating.R']
df_PD_graph = df_PD_graph.rename(columns={
    'Dose group':col_label, 
    'Time':x_label, 
    'Value':PD_y_label
}, errors="raise")
df_PD_graph = df_PD_graph.astype({col_label: 'category'}, errors="raise")

In [ ]:
import plotly.graph_objects as go
import plotly.colors

# Measure drug concentrations of patients
n_times = 1000
more_PK_times = np.linspace(start=0, stop=5, num=n_times)

fig = go.Figure()
colors = plotly.colors.qualitative.Plotly

params = individual_parameters[0,:]
params[2] = PK_actual_params[1]
avg_dose_amt = {}
for i, group in enumerate(dose_groups):
    # Set administration and dosing regimen
    measurements = df.loc[(df["Dose group"]==group)]
    dose = measurements["Dose"].dropna().mean()
    avg_dose_amt[group]=dose
    measurements = measurements.loc[(measurements["Observable"]=="PK_central.drug_concentration")]

    PKPD_model.set_dosing_regimen(dose=dose, start=0, period=0)
    result = PKPD_model.simulate(np.insert(params[:-2], 6, 0), more_PK_times)[0]

    # Visualise results
    fig.add_trace(go.Scatter(
        x=more_PK_times,
        y=result,
        mode='lines',
        line_color=colors[i],
        name='Sim. with dose '+str(group)+' '+dose_unit
    ))

    fig.add_trace(go.Scatter(
        x=measurements["Time"]-dose_time,
        y=measurements["Value"],
        mode='markers',
        marker_color=colors[i],
        name='Meas. with dose '+str(group)+' '+dose_unit
    ))

fig.update_layout(
    xaxis_title=x_label,
    yaxis_title=PK_y_label,
    template='plotly_white'
)
fig.update_yaxes(type="log", dtick=1)
fig.show()


# Measure drug concentrations of patients
n_times = 1000
more_PD_times = np.linspace(start=-48, stop=500, num=n_times)

fig = go.Figure()
for i, group in enumerate(dose_groups):
    # Set administration and dosing regimen
    measurements = df.loc[(df["Dose group"]==group)]
    dose = measurements["Dose"].dropna().mean()
    measurements = measurements.loc[(measurements["Observable"]=='circulating.R')]

    PKPD_model.set_dosing_regimen(dose=dose, start= dose_time, period=0)
    result = PKPD_model.simulate(np.insert(params[:-2], 6, 0), more_PD_times+dose_time)[1]

    # Visualise results
    fig.add_trace(go.Scatter(
        x=more_PD_times,
        y=result,
        mode='lines',
        line_color=colors[i],
        name='Sim. with dose '+str(group)+' '+dose_unit
    ))

    fig.add_trace(go.Scatter(
        x=measurements["Time"]-dose_time,
        y=measurements["Value"],
        mode='markers',
        marker_color=colors[i],
        name='Meas. with dose '+str(group)+' '+dose_unit
    ))

fig.update_layout(
    xaxis_title=x_label,
    yaxis_title=PD_y_label,
    template='plotly_white'
)
fig.update_yaxes(type="log", dtick=1)
fig.show()

In [ ]:
from plotly.subplots import make_subplots

# Sample individuals from population model
n_ids = 1000
individual_parameters = population_model.sample(
    parameters=PKPD_actual_pop_params,
    n_samples=n_ids,
    seed=1
)

# Visualise population model (parameter space)
fig = make_subplots(rows=3, cols=3)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 0],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 1],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 2],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=1,
    col=3
)
fig.add_trace(
    go.Scatter(
        name='Mean',
        x=np.exp([PKPD_actual_pop_params[2]]*2),
        y=[0, 0.03],
        mode='lines',
        line=dict(color='black', dash='dash'),
        showlegend=False
    ),
    row=1,
    col=3
)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 3],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=2,
    col=1
)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 4],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 5],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=2,
    col=3
)
# fig.add_trace(
#     go.Histogram(
#         name='Pop. model samples',
#         x=individual_parameters[:, 6],
#         histnorm='probability',
#         showlegend=False,
#         xbins_size=0.01,
#         marker_color='lightgrey'
#     ),
#     row=4,
#     col=1
# )
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 6],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=3,
    col=1
)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 7],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=3,
    col=2
)
fig.add_trace(
    go.Histogram(
        name='Pop. model samples',
        x=individual_parameters[:, 8],
        histnorm='probability',
        showlegend=False,
        xbins_size=0.01,
        marker_color='lightgrey'
    ),
    row=3,
    col=3
)

fig.update_layout(
    xaxis_title=problem.get_parameter_names()[0],
    xaxis2_title=problem.get_parameter_names()[1],
    xaxis3_title="PK_central.V_c",
    xaxis4_title=problem.get_parameter_names()[4],
    xaxis5_title=problem.get_parameter_names()[5],
    xaxis6_title=problem.get_parameter_names()[6],
    xaxis7_title=problem.get_parameter_names()[7],
    xaxis8_title=problem.get_parameter_names()[8],
    xaxis9_title=problem.get_parameter_names()[9],
    yaxis_title='Probability',
    yaxis3_title='Probability',
    yaxis5_title='Probability',
    xaxis_range=[PKPD_actual_pop_params[0]-0.1, PKPD_actual_pop_params[0]+0.1],
    xaxis2_range=[PKPD_actual_pop_params[1]-0.1, PKPD_actual_pop_params[1]+0.1],
    # xaxis3_range=[PKPD_actual_pop_params[2]-0.2, PKPD_actual_pop_params[2]+0.2],
    xaxis4_range=[PKPD_actual_pop_params[4]-0.1, PKPD_actual_pop_params[4]+0.1],
    xaxis5_range=[PKPD_actual_pop_params[5]-0.1, PKPD_actual_pop_params[5]+0.1],
    xaxis6_range=[PKPD_actual_pop_params[6]-0.1, PKPD_actual_pop_params[6]+0.1],
    xaxis7_range=[PKPD_actual_pop_params[7]-0.1, PKPD_actual_pop_params[7]+0.1],
    xaxis8_range=[PKPD_actual_pop_params[8]-0.1, PKPD_actual_pop_params[8]+0.1],
    xaxis9_range=[PKPD_actual_pop_params[9]-0.1, PKPD_actual_pop_params[9]+0.1],
    template='plotly_white',
    width=1000,
    height=750,
)
fig.show()

# fig.add_trace(
#     go.Histogram(
#         name='Pop. model samples',
#         x=individual_parameters[:, 10],
#         histnorm='probability',
#         showlegend=False,
#         xbins_size=0.01,
#         marker_color='lightgrey'
#     ),
#     row=5,
#     col=2
# )

In [ ]:
# Visualise population model (measurement space)
fig = make_subplots(rows=2, cols=1)
import plotly.express.colors as pxclrs

ind_colour_selection = np.asarray([
    # pxclrs.sample_colorscale(pxclrs.get_colorscale("turbo"), n_ids_data, low=0.8, high=1.0),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0, high=0.23),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0.35, high=0.65),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0.7, high=1.0)
])

for i, group in enumerate(dose_groups):

    # Simulate population distribution of PK measurements
    pop_measurements = np.empty(shape=(n_ids, len(more_PK_times)))
    dose = avg_dose_amt[group]
    PKPD_model.set_dosing_regimen(dose=dose, start=dose_time, period=0)
    
    if group!=0.0:
        for j, patient in enumerate(df_PK_graph.loc[df_PK_graph[col_label]==group]['ID'].unique()):
            x_data = df_PK_graph.loc[df_PK_graph['ID']==patient][x_label]-dose_time
            y_data = df_PK_graph.loc[df_PK_graph['ID']==patient][PK_y_label]
            fig.add_trace(
                go.Scatter(
                    x=x_data,
                    y=y_data,
                    mode='markers',
                    marker_color=ind_colour_selection[i, j],
                    name='Simulated data',
                    legendgroup = dose,
                    showlegend = False,
                ),
                row=1,
                col=1
            )

        # Plot 5th to 95th percentile of population distribution
        for idd, patient_parameters in enumerate(individual_parameters):
            result = PKPD_model.simulate(
                np.insert(patient_parameters[:-1], 6, 0),
                more_PK_times+dose_time
            )[0]
            pop_measurements[idd] = PK_noise_model.sample(
                patient_parameters[-2:-1], result)[:, 0]

        fifth = np.percentile(pop_measurements, q=5, axis=0)
        ninety_fifth = np.percentile(pop_measurements, q=95, axis=0)
        fig.add_trace(
            go.Scatter(
                x=np.hstack([more_PK_times, more_PK_times[::-1]]),
                y=np.hstack([fifth, ninety_fifth[::-1]]),
                line=dict(width=0, color=ind_colour_selection[i, int(n_ids_data/2)]),
                fill='toself',
                name='Population model',
                text=r"90% bulk probability",
                hoverinfo='text',
                showlegend=False
            ),
            row=1,
            col=1
        )
    
    for j, patient in enumerate(df_PD_graph.loc[df_PD_graph[col_label]==group]['ID'].unique()):
        x_data = df_PD_graph.loc[df_PD_graph['ID']==patient][x_label]-dose_time
        y_data = df_PD_graph.loc[df_PD_graph['ID']==patient][PD_y_label]
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=y_data,
                mode='markers',
                marker_color=ind_colour_selection[i, j],
                name='Simulated data',
                legendgroup = dose,
                showlegend = j==int(n_ids_data/2),
                legendgrouptitle = {'text': 'Dose '+str(group)+' '+dose_unit}
            ),
            row=2,
            col=1
        )
        
    # Plot 5th to 95th percentile of population distribution
    pop_measurements = np.empty(shape=(n_ids, len(more_PD_times)))
    for idd, patient_parameters in enumerate(individual_parameters):
        result = PKPD_model.simulate(np.insert(patient_parameters[:-1], 6, 0), more_PD_times+dose_time)[1]
        pop_measurements[idd] = PD_noise_model.sample(
            patient_parameters[-1:], result)[:, 0]
    fifth = np.percentile(pop_measurements, q=5, axis=0)
    ninety_fifth = np.percentile(pop_measurements, q=95, axis=0)
    fig.add_trace(
        go.Scatter(
            x=np.hstack([more_PD_times, more_PD_times[::-1]]),
            y=np.hstack([fifth, ninety_fifth[::-1]]),
            line=dict(width=0, color=ind_colour_selection[i, int(n_ids_data/2)]),
            fill='toself',
            name='Population model',
            text=r"90% bulk probability",
            hoverinfo='text',
            showlegend=True
        ),
        row=2,
        col=1
    )

fig.update_layout(
    xaxis_title=x_label,
    xaxis2_title=x_label,
    yaxis_title=PK_y_label,
    yaxis2_title=PD_y_label,
    width=750,
    height=750,
    template='plotly_white'
)

# y_range = [np.log10(np.min(df_graph[y_label]))-0.05, np.log10(np.max(more_values))+0.2]
# x_range = [0, 5]
fig['layout']['yaxis'].update(type="log", minor=dict(dtick='D1', showgrid=True), dtick = 1)
fig['layout']['xaxis'].update(dtick = 1)
fig['layout']['yaxis2'].update(dtick = 250)
fig['layout']['xaxis2'].update(dtick = 72, minor=dict(dtick=24, showgrid=True))
fig.write_image(image_file + "PKPD_"+drug+"_population_data.svg")
fig.show()

#### Maximum Log likelihood method

In [ ]:
from plotly import figure_factory as ff
from Code.Inference import OptimisationController

# Set the start point for optimisation
PD_start = np.exp(0.5*np.log(PD_bounds[0])+0.5*np.log(PD_bounds[1]))
PK_start = np.asarray(pandas.read_csv(PK_data_file + drug +"_opt_pop_results.csv")['Optimised'])
start_point = np.concatenate((PK_start, PD_start[:-2]), axis=0)[order]
start_point[6] = R_0_approx # Approximation of R_0

# Optimise the model with respect to the data
optimisation = OptimisationController(
    log_posterior
)
optimisation.set_n_runs(1)
optimisation.set_initial_point([1], [start_point])
optimisation.set_optimiser(pints.CMAES)
optimisation.set_transform(transformation)
result = optimisation.run(show_run_progress_bar=False, log_to_screen=False)

# Show summary of optimisation
print('Log-Likelihood Value: \t'+str(result['Score'].iloc[0]))
time = result['Time'].iloc[0]
print('Time Taken: \t'+str(int(time/60))+" minutes, "+str(int(time%60))+" seconds, ")

print('Result:')
parameters = (result.loc[result['ID'].isnull()])['Estimate'].values
summary_data = np.transpose(np.array([pop_param_names, parameters, PKPD_actual_pop_params]))
summary_df = pandas.DataFrame(summary_data, columns = ['Parameter', 'Optimised', 'True'])

summary_df.to_csv(PD_data_file+drug+"_opt_pop_results.csv", index=False)

opt_ind_params = np.asarray(result.dropna(subset=['ID'])['Estimate'])
np.save(PD_data_file+drug+"_opt_ind_results.npy", opt_ind_params)

summary_df = pandas.read_csv(PD_data_file+drug+"_opt_pop_results.csv")
table_df = summary_df.round({'Optimised':4, 'True':4})
table_df = table_df.set_index('Parameter').transpose()
fig =  ff.create_table(table_df, index=True)
fig.update_layout(
    width=900,
    height=65,
)
fig.write_image(image_file + "PKPD_"+drug+"_opt_table.svg")
fig.show()

In [ ]:
summary_df = pandas.read_csv(PD_data_file+drug+"_opt_pop_results.csv")
opt_ind_params = np.load(PD_data_file+drug+"_opt_ind_results.npy")

In [ ]:
df

In [ ]:
# Visualise the results
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express.colors as pxclrs

n_times = 1000
n_ids = 1000

ind_colour_selection = np.asarray([
    # pxclrs.sample_colorscale(pxclrs.get_colorscale("turbo"), n_ids_data, low=0.8, high=1.0),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0, high=0.23),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0.35, high=0.65),
    pxclrs.sample_colorscale(pxclrs.get_colorscale("viridis"), n_ids_data, low=0.7, high=1.0)
])

PK_time_span = df.loc[df["Observable"]=="PK_central.drug_concentration"]["Time"].max()
PD_time_span = df.loc[df["Observable"]=="circulating.R"]["Time"].max()
more_PK_times = np.linspace(
    start=0, stop=1.01*(PK_time_span-dose_time), num=n_times
)
more_PD_times = np.linspace(
    start=-dose_time, stop=1.01*(PD_time_span-dose_time), num=n_times
)

inferred_pop_params = np.asarray(summary_df['Optimised'])
avg_inferred = np.delete(inferred_pop_params, 3)
avg_inferred[2] = np.exp(avg_inferred[2])

sim_ind_params = population_model.sample(
    parameters=inferred_pop_params,
    n_samples=n_ids
)

# Plot the Observations
fig = make_subplots(rows=2, cols=1)

# Plot the population model for each dose
for i, group in enumerate(dose_groups):
    # Simulate and Plot from the population maximum likelihood estimation
    amt = dose_amts[i]
    PKPD_model.set_dosing_regimen(amt, start=dose_time, period=0)
    more_PK_values = PKPD_model.simulate(
        np.insert(avg_inferred[:-1], 6, 0),
        more_PK_times+dose_time
    )[0]
    fig.add_trace(
        go.Scatter(
            x=more_PK_times,
            y=more_PK_values,
            mode='lines',
            line=dict(color=ind_colour_selection[i, int(n_ids_data/2)], dash='dash'),
            name='Typical simulation',
            legendgroup= group,
            # legendgrouptitle = {'text': 'Dose '+str(group)+' '+dose_unit},
            showlegend=False
        ),
        row=1,
        col=1
    )
    more_PD_values = PKPD_model.simulate(np.insert(avg_inferred[:-2], 6, 0), more_PD_times)[1]
    fig.add_trace(
        go.Scatter(
            x=more_PD_times,
            y=more_PD_values,
            mode='lines',
            line=dict(color=ind_colour_selection[i, int(n_ids_data/2)], dash='dash'),
            name='Typical simulation',
            legendgroup= group,
            legendgrouptitle = {'text': 'Dose '+str(group)+' '+dose_unit},
            showlegend=True
        ),
        row=2,
        col=1
    )
    for j, patient in enumerate(df.loc[df['Dose group']==group]['ID'].unique()):
        PKPD_model.set_dosing_regimen(
            df.loc[(df['ID']==patient) & df['Observable'].isna()]['Dose'].iat[0], 
            start=dose_time,
            period=0
        )
        individual_parameters = avg_inferred.copy()
        individual_parameters[2] = opt_ind_params[i*n_ids_data + j]
        if group!=0.0:
            individual_PK_values = PKPD_model.simulate(
                np.insert(individual_parameters[:-2], 6, 0),
                more_PK_times+dose_time
            )[0]
            fig.add_trace(
                go.Scatter(
                    x=more_PK_times,
                    y=individual_PK_values,
                    name='Individual simulation',
                    legendgroup = group,
                    showlegend= False,
                    mode="lines",
                    line=go.scatter.Line(color=ind_colour_selection[i, j], width=0.5),
                    opacity=0.5,
                ),
                row=1,
                col=1
            )
            x_data = df_PK_graph.loc[df_PK_graph['ID']==patient][x_label]-dose_time
            y_data = df_PK_graph.loc[df_PK_graph['ID']==patient][PK_y_label]
            fig.add_trace(go.Scatter(
                x=x_data,
                y=y_data,
                mode='markers',
                marker_color=ind_colour_selection[i, j],
                name='Data',
                legendgroup = group,
                showlegend = False,
            ))
        individual_PD_values = PKPD_model.simulate(
            np.insert(individual_parameters[:-2], 6, 0),
            more_PD_times+dose_time
        )[1]
        fig.add_trace(
            go.Scatter(
                x=more_PD_times,
                y=individual_PD_values,
                name='Individual simulation',
                legendgroup = group,
                showlegend= j==int(n_ids_data/2),
                mode="lines",
                line=go.scatter.Line(color=ind_colour_selection[i, j], width=0.5),
                opacity=0.5,
            ),
            row=2,
            col=1
        )
        x_data = df_PD_graph.loc[df_PD_graph['ID']==patient][x_label]-dose_time
        y_data = df_PD_graph.loc[df_PD_graph['ID']==patient][PD_y_label]
        fig.add_trace(
            go.Scatter(
                x=x_data,
                y=y_data,
                mode='markers',
                marker_color=ind_colour_selection[i, j],
                name='Data',
                legendgroup = group,
                showlegend = j==int(n_ids_data/2),
            ),
            row=2,
            col=1
        )
        
    PKPD_model.set_dosing_regimen(amt, start=dose_time, period=0)
    if group!=0.0:
        # Find 5th to 95th percentile of population distribution
        simulations = np.empty(shape=(n_ids, n_times))
        for idd, patient_parameters in enumerate(sim_ind_params):
            ind_sim = PKPD_model.simulate(
                np.insert(patient_parameters[:-2], 6, 0),
                more_PK_times+dose_time
            )[0]
            simulations[idd] = PK_noise_model.sample(
                patient_parameters[-2:-1], ind_sim)[:, 0]
        fifth = np.percentile(simulations, q=5, axis=0)
        ninety_fifth = np.percentile(simulations, q=95, axis=0)

        # Plot the variability
        fig.add_trace(
            go.Scatter(
                x=np.hstack([more_PK_times, more_PK_times[::-1]]),
                y=np.hstack([fifth, ninety_fifth[::-1]]),
                line=dict(
                    width=0,
                    color=ind_colour_selection[i, int(n_ids_data/2)]
                ),
                fill='toself',
                name='Population model',
                text=r"90% bulk probability",
                hoverinfo='text',
                legendgroup= group,
                showlegend=False
            ),
            row=1,
            col=1
        )
    # Find 5th to 95th percentile of population distribution
    simulations = np.empty(shape=(n_ids, n_times))
    for idd, patient_parameters in enumerate(sim_ind_params):
        ind_sim = PKPD_model.simulate(
            np.insert(patient_parameters[:-2], 6, 0),
            more_PD_times + dose_time
        )[1]
        simulations[idd] = PD_noise_model.sample(
            patient_parameters[-1:], ind_sim)[:, 0]
    fifth = np.percentile(simulations, q=5, axis=0)
    ninety_fifth = np.percentile(simulations, q=95, axis=0)

    # Plot the variability
    fig.add_trace(
        go.Scatter(
            x=np.hstack([more_PD_times, more_PD_times[::-1]]),
            y=np.hstack([fifth, ninety_fifth[::-1]]),
            line=dict(
                width=0,
                color=ind_colour_selection[i, int(n_ids_data/2)]
            ),
            fill='toself',
            name='Population model',
            text=r"90% bulk probability",
            hoverinfo='text',
            legendgroup= group,
            showlegend=False
        ),
        row=2,
        col=1
    )

fig.update_layout(
    xaxis_title=x_label,
    xaxis2_title=x_label,
    yaxis_title=PK_y_label,
    yaxis2_title=PD_y_label,
    width=750,
    height=750,
    template='plotly_white'
)

fig['layout']['yaxis'].update(type="log", minor=dict(dtick='D1', showgrid=True), dtick = 1)
fig['layout']['xaxis'].update(dtick = 1)
fig['layout']['yaxis2'].update(dtick = 250)
fig['layout']['xaxis2'].update(dtick = 72, minor=dict(dtick=24, showgrid=True))
fig.write_image(image_file + "PKPD_"+drug+"_opt_graph.svg")
fig.show()

#### Bayesian Inference

In [ ]:
# Further adjustables
num_iterations = 2000  # max iterations per parameter
num_samples = 500  # number of wanted final samples per parameter per run
n_runs = 5

In [ ]:
from Code.Inference import SamplingController

inferred_pop_params = pandas.read_csv(PD_data_file+drug+"_opt_pop_results.csv")
opt_params = inferred_pop_params['Optimised']
opt_ind_params = np.load(PD_data_file+drug+"_opt_ind_results.npy")

pop_ranges = np.asarray([
    np.concatenate((PK_bounds[0,0:1], [np.log(PK_bounds[0,1])], [np.exp(-1-2*0.4)], PK_bounds[0,2:], PD_bounds[0])),
    np.concatenate((PK_bounds[1,0:1], [np.log(PK_bounds[1,1])], [np.exp(-1+2*0.4)], PK_bounds[1,2:], PD_bounds[1]))
])[:, order]
start_pop_points = np.concatenate((
    np.linspace(start=opt_params, stop=pop_ranges[0], num=4*(n_runs-1), endpoint=False)[:int(0.5*(n_runs-1))], 
    np.linspace(start=opt_params, stop=pop_ranges[1], num=4*(n_runs-1), endpoint=False)[:int(0.5*(n_runs-1))], 
))

# for i, param in enumerate(population_model.get_parameter_names()):
#     print(param, start_pop_points[:, i])
# print(sampler._initial_params)
# start_points = optimisation.make_initial_point(3, start_pop_points)
# start_points = np.concatenate((
#     start_points, optimisation.make_initial_point(1, opt_params, opt_ind_params)
# ))
sampler = SamplingController(log_posterior)
sampler.set_n_runs(n_runs)
# if n_runs%2==0:
#     sampler.set_initial_point(range(1, n_runs), start_pop_points[1:, :])
# else:
#     sampler.set_initial_point(range(1, n_runs), start_pop_points)
sampler.set_initial_point([n_runs], opt_params, opt_ind_params)

ini_point = sampler._initial_params
# print(ini_point)
for i, param in enumerate(population_model.get_parameter_names()):
    print(param, ini_point[:, i+n_ids_data*len(dose_amts)])

In [ ]:
# cov_0 = 0.02 * np.abs(np.concatenate((opt_ind_params, opt_params))) # np.mean(np.abs(sampler._initial_params), axis = 0)
cov_0 = None
sampler.set_sampler(pints.NoUTurnMCMC)
sampler.set_stop_criterion(max_iterations= num_iterations*len(opt_params), r_hat=1.01) # num_iterations*len(opt_params)
# sampler.set_transform(transformation)
posterior_samples = sampler.run(n_iterations=num_samples*len(opt_params), sigma0=cov_0)

In [ ]:
import xarray as xr
# posterior_samples.to_netcdf(PD_data_file+drug+"_MCMC_samples.nc")
posterior_samples = xr.open_dataset(PD_data_file+drug+"_MCMC_samples.nc").load()

In [ ]:
posterior_samples

In [ ]:
import arviz as az
import matplotlib.pyplot as plt

az.style.use(["arviz-white",  "arviz-viridish"])

lines = list(zip(*(
    population_model.get_parameter_names(),
    [{}]*len(PKPD_actual_pop_params),
    PKPD_actual_pop_params)))
actual_ind_Vc = np.load(PD_data_file + drug +"_ind_Vc_param.npy")
lines = lines+[(
    "central.V_c",
    dict(zip(
        ['individual']*(n_ids_data*len(dose_amts)),
        df['ID'].unique()
    )),
    actual_ind_Vc
)]
# az.plot_trace(posterior_samples, var_names=('^(?!.*?central.V_c).*'), filter_vars="regex", lines=lines, divergences='bottom')
# plt.savefig(image_file +"PK_"+drug+"MCMC_Pop_trace_plot.svg")
# plt.show()

In [ ]:
PKPD_actual_pop_params

In [ ]:
lines[2] = (
    population_model.get_parameter_names()[2],
    {},
    np.exp(PKPD_actual_pop_params[2])
)
def graph_tansform(params_to_trans):
    trans_param = params_to_trans
    try:
        trans_param['Log mean '+param] = np.exp(trans_param['Log mean '+param])
    except:
        pass
    return trans_param
for param in population_model.get_parameter_names():
    az.plot_trace(posterior_samples, var_names=('.*'+param), filter_vars="regex", lines=lines, transform=graph_tansform)
    param_name = param.split('.')[-1]
    plt.savefig(image_file +"PK_"+drug+"MCMC_"+param_name+"_trace_plot.svg")
    plt.show()

In [ ]:
posterior_samples.close()